#Definiciones y carga inicial

In [ ]:
import requests
from google.colab import files
import pandas as pd
from io import StringIO
import json

API_ROOT = 'https://servicios.ine.es/wstempus/js'

pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

def consulta(funcion='OPERACIONES_DISPONIBLES', input=None, params=None, idioma='es'):
    if input:
        query_list = [API_ROOT, idioma, funcion, input]
    else:
        query_list = [API_ROOT, idioma, funcion]
    url = "/".join(query_list)
    if params:
        url = '{}?{}'.format(url, "&".join(params))
    print("Consulta realizada: {}".format(url))
    response = requests.get(url)
    if response.content.decode('utf-8') == '':
        print("La consulta no ha devuelto nada.")
        return None
    else:
        return response.json()

def titulo(texto):
    print("")
    print(texto.upper())
    print("=" * 50)

def muestra(datos, orden=None):
    if 'Nombre' in datos:
        titulo("Muestra de {}".format(datos['Nombre']))
        datos = datos['Data']
    if orden:
        datos.sort(key=lambda x: x[orden])
    df = pd.read_json(StringIO(json.dumps(datos)))
    print(df.head(100))

def muestra_todos(datos, orden=None):
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    muestra(datos, orden)
    pd.set_option('display.max_rows', 10)
    pd.set_option('display.max_columns', 5)

def exporta(datos):
    if 'Data' in datos:
        filename = '{} - {}.csv'.format(datos['COD'], datos['Nombre'])
        datos = datos['Data']
    else:
        filename = 'metadatos.csv'
    df = pd.read_json(StringIO(json.dumps(datos)))
    df.to_csv(filename)
    files.download(filename)

#Ejemplos

Las consultas siguen el mismo patrón que se indica en la [documentación de la API](https://www.ine.es/dyngs/DataLab/manual.html?cid=47).

- **funcion**: para la descarga de datos habitualmente será DATOS_SERIE o DATOS_TABLA.
- **input**: el nombre de la serie o tabla cuyos datos queremos descargar.
- **parametros**: algunas consultas requieren parámetros específicos. En el caso de la consulta de datos, además de los comunes podemos usar date y nult. Ver la documentación para mayor información.

###Consulta y descarga de una serie de datos

In [ ]:
funcion = 'DATOS_SERIE'
input = 'IPC251839'
parametros = [
    'nult=100'
]

datos = consulta(funcion=funcion, input=input, params=parametros)
if datos:
    muestra(datos)
    exporta(datos)

Consulta realizada: https://servicios.ine.es/wstempus/js/es/DATOS_SERIE/IPC251839?nult=100

MUESTRA DE TOTAL NACIONAL. BIENES COVID-19. VARIACIÓN MENSUAL. 
            Fecha  FK_TipoDato  ...  Valor  Secreto
0   1483225200000            1  ...    0.6    False
1   1485903600000            1  ...    0.1    False
2   1488322800000            1  ...   -0.2    False
3   1490997600000            1  ...   -0.1    False
4   1493589600000            1  ...    0.2    False
..            ...          ...  ...    ...      ...
47  1606777200000            1  ...   -0.3    False
48  1609455600000            3  ...    0.7    False
49  1612134000000            3  ...    0.0    False
50  1614553200000            3  ...    0.0    False
51  1617228000000            3  ...    0.3    False

[52 rows x 6 columns]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

###Ver operaciones disponibles

In [ ]:
funcion = 'OPERACIONES_DISPONIBLES'
input = ''

operaciones = consulta(funcion, input)
if operaciones:
    titulo("Operaciones disponibles:")
    muestra_todos(operaciones, 'Codigo')


###Ver las series de una operación

In [ ]:
funcion = 'SERIES_OPERACION'
input = 'MYH'
parametros = [
    "page=1"
]

series = consulta(funcion, input, parametros)
if series:
    titulo("Series disponibles:")
    muestra_todos(series)
    exporta(series)

###Ver los datos de la serie anterior

In [ ]:
funcion = 'DATOS_SERIE'
input = 'MYH19792'
parametros = [
    "page=1"
]

series = consulta(funcion, input, parametros)
if series:
    titulo("Series disponibles:")
    muestra_todos(series)

Consulta realizada: https://servicios.ine.es/wstempus/js/es/DATOS_SERIE/MYH19792?page=1
La consulta no ha devuelto nada.
